In [1]:
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT * FROM sqlite_master
WHERE type = 'table';

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


# Query to find out which genres sell the most tracks in the USA

In [146]:
%%sql

WITH 
    genre_usa AS
        (
        SELECT 
            genre.name genre,
            COUNT(quantity) number,
            track.album_id
        FROM track
        INNER JOIN genre on genre.genre_id = track.genre_id
        INNER JOIN invoice_line on invoice_line.track_id = track.track_id
        INNER JOIN invoice on invoice.invoice_id = invoice_line.invoice_id
        WHERE billing_country = 'USA'
        GROUP BY genre.name
        ORDER BY 2 DESC
        ),
    genre_usa_percent AS
        (
        SELECT
            genre,
            album_id,
            number,
            ROUND(CAST(number AS Float) / CAST((select sum(number) from genre_usa) AS Float)* 100, 2) percent
        FROM genre_usa
        )

SELECT
    genre,
    number 'number of tracks sold in the USA',
    percent 'percentage_sold(%)'
FROM genre_usa_percent
LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,number of tracks sold in the USA,percentage_sold(%)
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


From the analysis above, the top three genre with the most tracks sold are;
- Rock
- Alternative & Punk
- Metal

# Query that finds the total dollar amount of sales assigned to each sales support agent within the company

In [201]:
%%sql

WITH
    inv_cust AS
        (
        SELECT 
            support_rep_id,
            COUNT(*) number_of_sale,
            SUM(total) cost
        FROM customer
        INNER JOIN invoice ON invoice.customer_id = customer.customer_id
        GROUP BY support_rep_id
        ),
    employee_sales AS
        (
        SELECT * FROM employee
        INNER JOIN inv_cust ON inv_cust.support_rep_id = employee_id
        )

SELECT 
    employee_sales.first_name || ' ' || employee_sales.last_name employee_name,
    title,
    hire_date,
    number_of_sale,
    ROUND(cost, 2) 'cost_of_sale($)'
FROM employee_sales

 * sqlite:///chinook.db
Done.


employee_name,title,hire_date,number_of_sale,cost_of_sale($)
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,212,1731.51
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,214,1584.0
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,188,1393.92


Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase

As it's shown in the table above, the sales support department of the company made (212+214+188) total sales which makes a sum of (1731.51+1584.0+1393.92)$

i.e 614 sales ----> **$4709.43**

Jane Peacock made the highest number of sale accumulatiing $1731.51 and this is majorly due to the hire date. It can be inferred from the table that the number of sale and cost of sale have a direct relationship with the **hire date** (i.e the sales agent hired the earliest brought in the largest sale.)

# Query that returns the number of customers for each country
NB: Countries with just one customer are collated into 'Other'

In [88]:
%%sql

WITH
    count_query AS
        (
        SELECT
            CASE
                WHEN COUNT(*) = 1 THEN 'Other'
                ELSE country
            END AS country,
            COUNT(*) number_of_customer
        FROM customer
        GROUP BY country
        ),
    soln_table AS
        (
        SELECT 
            country,
            SUM(number_of_customer) number_of_customer
        FROM count_query
        GROUP BY country
        )

SELECT 
    country, 
    number_of_customer 
FROM 
    (
    SELECT
        soln_table.*,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM soln_table
    )
ORDER BY sort 

 * sqlite:///chinook.db
Done.


country,number_of_customer
Brazil,5
Canada,8
Czech Republic,2
France,5
Germany,4
India,2
Portugal,2
USA,13
United Kingdom,3
Other,15
